In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!unzip /kaggle/input/sberbank-russian-housing-market/train.csv.zip

## Reading data

In [ ]:
dataset = pd.read_csv(r'./train.csv')
pd.set_option('display.max_columns', 300)
dataset.head()

In [ ]:
dataset.shape

# Descriptive Statistics

In [ ]:
pd.set_option('display.max_rows', 300)
dataset.isnull().sum()

In [ ]:
dataset.dtypes

In [ ]:
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset['year'] = dataset['timestamp'].dt.year

In [ ]:
dataset.drop(['id','timestamp','sub_area'],axis=1,inplace=True)

In [ ]:
numerical_feature = [feature for feature in dataset.columns if dataset[feature].dtypes != "O"]
numerical_feature


In [ ]:
for feature in numerical_feature:
    print("{} has null values :   {}".format(feature,dataset[feature].isnull().sum()))

In [ ]:
categorical_feature = [feature for feature in dataset.columns if dataset[feature].dtypes == "O"]
categorical_feature

In [ ]:
for feature in categorical_feature:
    print("{} has categorical features :   {}".format(feature,len(dataset[feature].unique())))

In [ ]:
for feature in categorical_feature:
    print("{} has null values :   {}".format(feature,dataset[feature].isnull().sum()))

In [ ]:
continuous_feature = [feature for feature in numerical_feature if len(dataset[feature].unique()) > 40]
continuous_feature

In [ ]:
discrete_feature = [feature for feature in numerical_feature if len(dataset[feature].unique()) <= 40]
discrete_feature

In [ ]:
for feature in discrete_feature:
    print("{} feature has null values : {}".format(feature,dataset[feature].isnull().sum()))

##  Univariate/Bivariate Analysis

In [ ]:
for feature in categorical_feature:
    sns.barplot(x = dataset[feature],y = dataset['price_doc'])
    plt.xlabel(feature)
    plt.ylabel('price_doc')
    plt.show()


In [ ]:
for feature in discrete_feature:
    sns.barplot(x = dataset[feature],y = dataset['price_doc'])
    plt.xlabel(feature)
    plt.ylabel('price_doc')
    plt.show()


In [ ]:
for feature in continuous_feature:
    dataset[feature].hist(bins=30)
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.show()

In [ ]:
missing_column = [feature for feature in dataset.columns if dataset[feature].isnull().sum() > 0 ]
missing_column

In [ ]:
for feature in missing_column:
    sns.boxplot(dataset[feature])
    plt.xlabel(feature)
    plt.show()

In [ ]:
dataset['build_year']

In [ ]:
missing_column_numerical = [feature for feature in numerical_feature if dataset[feature].isnull().sum() > 0 ]
missing_column_numerical

In [ ]:
missing_column_discrete = [feature for feature in numerical_feature if dataset[feature].isnull().sum() > 0 ]
missing_column_discrete

# Feature Engineering

In [ ]:
for feature in missing_column_numerical:
    value = dataset[feature].median()
    dataset[feature].fillna(value,inplace=True)

In [ ]:
for feature in missing_column_discrete:
    value = dataset[feature].mode()
    dataset[feature].fillna(value,inplace=True)

In [ ]:
dataset.isnull().sum()

In [ ]:
for feature in continuous_feature:
    if 0 in dataset[feature].unique():
        pass
    else:
        dataset[feature] = np.log(dataset[feature])
        dataset[feature].hist(bins=30)
        plt.xlabel(feature)
        plt.ylabel('Count')
        plt.show()

In [ ]:
dataset.describe()

In [ ]:
import matplotlib.pyplot as plt 
plt.rcParams["figure.figsize"] = (20,3)
sns.heatmap(dataset.corr(),annot= True,cmap= 'coolwarm')

In [ ]:
data = dataset.drop("price_doc",axis=1)

In [ ]:
def correlation(dataset,threshold):
    col_corr = set()
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i,j] >= threshold):
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
correlation_features = correlation(dataset,0.8)
correlation_features

In [ ]:
dataset.drop(correlation_features,axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
categorical_feature

In [ ]:
feat = dataset[categorical_feature]
feat

In [ ]:
for feature in categorical_feature:
    if feature == 'ecology':
        pass
    else:
        dataset[feature] = pd.get_dummies(dataset[feature],drop_first=True)

In [ ]:
dataset

In [ ]:
dataset.isnull().sum()

In [ ]:
cor = dataset.corr()
cor

In [ ]:
a = abs(cor['price_doc'])
result = abs(a[a>0.5])
result

In [ ]:
imp_feature = []
for i in range(0,len(result)):
  if result.iloc[i] > 0.1:
    print(result[i])
    imp_feature.append(result[i])

In [ ]:
len(imp_feature)

In [ ]:
data = dataset.copy()

In [ ]:
X = data.drop('price_doc',axis=1)
Y = data['price_doc']

In [ ]:
X.drop('ecology',inplace=True,axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=50)

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

# fit model to training data
xgb_model = XGBRegressor(n_estimators=100)
xgb_model.fit(X_train, Y_train)

print("Feature Importances : ", xgb_model.feature_importances_)

# plot feature importance
plt.rcParams["figure.figsize"] = (20,50)
plot_importance(xgb_model)
plt.show()

In [ ]:
sorted_idx = np.argsort(xgb_model.feature_importances_)[::-1]

In [ ]:
temp = []
for index in sorted_idx:
    print([X_train.columns[index], xgb_model.feature_importances_[index]])
    temp.append([X_train.columns[index], xgb_model.feature_importances_[index]])

In [ ]:
temp

In [ ]:
not_imp_feature = []
for i in range(50,len(temp)):
  not_imp_feature.append(temp[i][0])

In [ ]:
len(not_imp_feature)

In [ ]:
X_train.drop(not_imp_feature,axis=1,inplace=True)

In [ ]:
X_test.drop(not_imp_feature,axis=1,inplace=True)

In [ ]:
X_test.shape

In [ ]:
X_train.to_csv('train_data.csv')
X_test.to_csv('test_data.csv')

In [ ]:
Y_train.to_csv('ytrain_data.csv')
Y_test.to_csv('ytest_data.csv')
